In [1]:
import os
import numpy as np
import pandas as pd
from scipy import signal as sig
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

c:\Users\Alli Ajagbe\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
ddir = os.path.join('.','precipitation')
pr = xr.open_mfdataset(f'{ddir}/*.nc')